#  Convergence of Heavyball, Comparisons

In this discussion, we will explore
* Convergence rate of heavyball
* Comparisons with other methods

---

Last time we introduced **heavyball method**, also referred to as **steepest descent with momentum**, defined by the following:

$$ \begin{align*}
    \mathbf{p}_k &= -\alpha_k\nabla f_k + \beta \mathbf{p}_{k-1} \\
    \mathbf{x}_{k+1} &= \mathbf{x}_k + \mathbf{p}_k
\end{align*} $$

where $\beta\in(0,1)$ is called the *momentum parameter*. We showed an example of this method improving upon the convergence of steepest descent, but we didn't really give a mathematical justification for this. That is our goal today, but first let us recall a few facts about steepest descent.

## Convergence rate of steepest descent

We showed previously that, for quadratic functions, the fixed step steepest descent algorithm admits the inequality

$$ \frac{\|\mathbf{x}_{k+1}-\mathbf{x}^*\|}{\|\mathbf{x}_{k}-\mathbf{x}^*\|} \le \frac{\lambda_{max}(Q)-\lambda_{min}(Q)}{\lambda_{max}(Q) + \lambda_{min}(Q)} \equiv \frac{\kappa-1}{\kappa+1} $$

provided $\alpha$ is chosen optimally, where $\kappa\equiv \frac{\lambda_{max}(Q)}{\lambda_{min}(Q)}\ge 1$ is the condition number of $Q$. This inequality demonstrates the linear convergence of steepest descent for quadratic functions since if $\kappa>1$, $\frac{\kappa-1}{\kappa+1} = 1 - \frac{2}{\kappa+1}<1$. Recall also that this bound is  achieved by choosing

$$\alpha = \frac{2}{\lambda_{max}(Q) + \lambda_{min}(Q)} $$

We remarked that this result could be extended to certain non-quadratic functions, provided they are "nice" enough. In fact, the strict requirement is that $f$ be a $C^2$ function that is **strongly convex** and **strongly smooth**. A function is strongly convex if

$$ f(\mathbf{y}) \ge f(\mathbf{x}) +\nabla f(\mathbf{x})^T(\mathbf{y}-\mathbf{x}) + \frac{\mu}{2}\|\mathbf{y}-\mathbf{x}\|^2 $$

holds for all $\mathbf{x},\mathbf{y}\in\mathbb{R}^n$ and some constant $\mu > 0$ (strictly!). A similar expression exists for strongly smooth, i.e. 

$$ f(\mathbf{y}) \le f(\mathbf{x}) +\nabla f(\mathbf{x})^T(\mathbf{y}-\mathbf{x}) + \frac{L}{2}\|\mathbf{y}-\mathbf{x}\|^2 $$

for some $L\ge\mu$. The combination of these two inequalities are essentially putting strict upper ($L$) and lower ($\mu$) bounds on the eigenvalues of the Hessian, rather than allowing arbitrarily small or large values (thus the "strong" qualifier). If $f$ is quadratic, $L=\lambda_{max}(Q)$ and $\mu=\lambda_{min}(Q)$, i.e. all quadratic functions are strongly convex/smooth. Provided the bounds of $L$ and $\mu$ are able to be determined for a non-quadratic function (which may be difficult in practice), the result above generalizes nicely: choosing

$$ \alpha = \frac{2}{L+\mu} \implies \frac{\|\mathbf{x}_{k+1}-\mathbf{x}^*\|}{\|\mathbf{x}_{k}-\mathbf{x}^*\|} \le \frac{L-\mu}{L + \mu} $$

and so we still get linear convergence since $L\ge\mu$.

## Convergence rate of Heavyball

If $L\gg\mu$ above, the constant $r\equiv \frac{L-\mu}{L+\mu}$ is very close to 1, which can be interpreted as the error not improving very much each iteration, resulting in slow convergence. It is shown in lecture by an argument based on treating the iteration update as a [fixed point iteration](https://en.wikipedia.org/wiki/Fixed-point_iteration) that a very similar bound can be obtained for the heavyball update. If we choose

$$ \alpha = \frac{4}{(L^{1/2} + \mu^{1/2})^2},\quad \beta = \frac{(L^{1/2} - \mu^{1/2})^2}{(L^{1/2} + \mu^{1/2})^2} \qquad \implies\qquad  \frac{\|\mathbf{x}_{k+1}-\mathbf{x}^*\|}{\|\mathbf{x}_{k}-\mathbf{x}^*\|} \le \frac{L^{1/2}-\mu^{1/2}}{L^{1/2} + \mu^{1/2}} $$

which is the same as the above with the replacement $L\to L^{1/2}$, $\mu\to\mu^{1/2}$. For the same values of $L,\mu$, the ratio $r$ for heavyball is usually smaller than the ratio $r$ for steepest descent, resulting in faster convergenge. Note that the convergence for heavyball is **still linear**, but since the ratio of consecutive errors is smaller, the iteration will converge more quickly.

We demonstrate this difference below by comparing steepest descent to heavyball with these optimal fixed parameters on the (quadratic) function we looked at last time:

$$ f(x,y) = x^2 + 10y^2 $$

which has $L=20$, $\mu=2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

f = lambda x,y: x**2 + 10*y**2
df = lambda x,y: np.array([2*x, 20*y])
Q = np.array([[2,0],[0,20]])

In [ ]:
%%time
x = np.array([10,1])
path_SD = [x]
tol = 1e-8
alpha = 2/(20+2)  # optimal step size
max_steps = 1000
dx = df(x[0],x[1])
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    xnew = x - alpha*dx
    path_SD.append(xnew)
    x = xnew
    i += 1
    dx = df(x[0],x[1])
    
path_SD=np.array(path_SD)
print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')

In [ ]:
def plot_path(path, func, title, window=[-10,10,-5,5], contours=50, skip=1):
    '''Plots path defined in (N,2) array "path" on a contour plot of "func" in window "window"'''
    plt.figure(figsize=(10,5),dpi=150)
    X = np.linspace(window[0],window[1],300)
    Y = np.linspace(window[2],window[3],300)
    Xmesh, Ymesh = np.meshgrid(X,Y)
    Z = func(Xmesh,Ymesh)
    CS = plt.contour(Xmesh, Ymesh, Z, contours, cmap='jet')
    plt.clabel(CS,inline_spacing=0,fmt='%d')
    plt.axis(window)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)

    for i in range(path.shape[0]-1): # iterate through steps
        if i%skip==0:
            # only plot arrows every "skip" iterations
            plt.arrow(path[i,0],path[i,1],path[i+1,0]-path[i,0],path[i+1,1]-path[i,1],
                      color='k',length_includes_head=True)
    plt.show()

In [ ]:
plot_path(path_SD,f,'Steepest descent', contours=np.arange(1,20)**2)

We see this takes quite a few iterations even for a quadratic function. If we investigate the ratio

$$ r = \frac{L-\mu}{L+\mu} = \frac{20-2}{20+2} \approx 0.81818 $$

we see the error in each iteration is still more than $80\%$ of the previous iteration, quite large. We write a function below to determine this value:

In [ ]:
def approx_error_ratio(path, minimizer, numToAvg=10, showPlot=False):
    '''Given a path defined by an iteration and a known minimizer, approximates the limit of the ratio of errors'''
    err = np.linalg.norm(path-np.array(minimizer),axis=1) # ||x_k-x*||=e_k
    
    # if converged in very few steps, return infinite order
    if len(err)<=2:
        return np.inf
    
    rr = np.zeros(len(err)-2)
    for i in range(len(err)-2):
        rr[i] = err[i+1]/err[i]
    
    if numToAvg>len(rr):
        # if not enough iterations to average, just average all
        r = np.mean(rr)
    else:
        # return mean of last few iterations
        r = np.mean(rr[-numToAvg:])
        
    # plot
    if showPlot:
        plt.plot(rr)
        plt.plot(rr*0+r)
        plt.xlabel('k')
        plt.ylabel('r')
        plt.title(f'r={r}')
        plt.show()
        
    return r, rr

In [ ]:
r_SD, rr_SD = approx_error_ratio(path_SD,[0,0],showPlot=True)
print('r =',r_SD)

And now doing the same using heavyball, we should expect faster convergence, since

$$ r = \frac{L^{1/2}-\mu^{1/2}}{L^{1/2}+\mu^{1/2}} \approx 0.51949 $$

meaning each consecutive error is only about $52\%$ of the previous error.

In [ ]:
%%time
x = np.array([10,1])
path_HB = [x]
tol = 1e-8
beta = (np.sqrt(20)-np.sqrt(2))**2/(np.sqrt(20)+np.sqrt(2))**2  # optimal momentum parameter
alpha = 4/(np.sqrt(20)+np.sqrt(2))**2                           # optimal step size
max_steps = 1000
dx = df(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    pk = -alpha*dx + beta*pk
    xnew = x + pk
    path_HB.append(xnew)
    x = xnew
    i += 1
    dx = df(x[0],x[1])

path_HB = np.array(path_HB)
print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')

In [ ]:
plot_path(path_HB,f,'Heavyball', contours=np.arange(1,20)**2)

In [ ]:
r_HB, rr_HB = approx_error_ratio(path_HB,[0,0],showPlot=True)
print('r =',r_HB)

In [ ]:
print(rr_HB[-5:])

Of course, the above is a very ideal example where the values of $L$ and $\mu$ are easily calculable since the function is quadratic. If we apply a more sophisticated method such as Newton's, we expect convergence in one step. Even conjugate gradient, which is very similar to heavyball (indeed it can be described as heavyball with "adaptive" parameters $\alpha$ and $\beta$), is expected to converge in at most two steps since this is a function in $\mathbb{R}^2$. We show this below, choosing the Fletcher-Reeves (FR) conjugate gradient step, recognizing that for quadratic functions, this just simplifies to the linear conjugate gradient method:

In [ ]:
def FR_beta(Dfk,Dfk1):
    return (Dfk1@Dfk1) / (Dfk@Dfk)

In [ ]:
%%time
x = np.array([10,1])
path_FR = [x]
tol = 1e-8
max_steps = 1000
dx = df(x[0],x[1])
pk = -dx    # first step is steepest descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:    
    alpha = -(pk@dx)/(pk@(Q@pk)) # use optimal alpha
    xnew = x + alpha*pk
    dx1 = df(xnew[0],xnew[1])
    bk = FR_beta(dx,dx1)
    if i>0 and i%3 ==0:
        bk = 0    # restarting
    pk = -dx1 + bk*pk

    path_FR.append(xnew)
    x = xnew
    dx = dx1
    i += 1

path_FR=np.array(path_FR)
print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')

In [ ]:
plot_path(path_FR,f,'Conjugate gradient', contours=np.arange(1,20)**2)

---

## Non-quadratic example

Since conjugate gradient and heavyball are so similar, an interesting question (which you will explore in homework) is which performs better on non-quadratic functions. Just to spice things up, let us choose a different non-quadratic function than we usually choose, the [Beale function](http://benchmarkfcns.xyz/benchmarkfcns/bealefcn.html), defined by

$$ f(x,y) = (1.5-x+xy)^2 + (2.25-x+xy^2)^2 + (2.625-x+xy^3)^2 $$

This function has a global minimum at $(3,0.5)$, but it has several non-ideal features like as saddle points ($\nabla f=\mathbf{0}$ but not a minimum), local suboptimal minima ($\nabla f = \mathbf{0}$ but better solutions exist), and a very flat plateau ($\nabla f\approx\mathbf{0}$ so steps are small) near the global minimum. Let's make a contour plot of this function below to investigate:

In [ ]:
g = lambda x, y: (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

# numerical approximation of gradient using central difference; aint nobody got time for that
h = 1e-6
dg = lambda x, y: np.array([0.5*(g(x+h, y)-g(x-h, y))/h,
                            0.5*(g(x, y+h)-g(x, y-h))/h])

In [ ]:
from matplotlib.colors import LogNorm  # useful for visualizing this function

def plot_path_Beale(path, func, title, window=[-4.5,4.5,-4.5,4.5], contours=np.logspace(-2,6,100), skip=1):
    '''Plots path defined in (N,2) array "path" on a contour plot of "func" in window "window"'''
    plt.figure(figsize=(5,5),dpi=150)
    X = np.linspace(window[0],window[1],300)
    Y = np.linspace(window[2],window[3],300)
    Xmesh, Ymesh = np.meshgrid(X,Y)
    Z = func(Xmesh,Ymesh)
    CS = plt.contourf(Xmesh, Ymesh, Z, contours, cmap='jet', norm = LogNorm())
    plt.axis(window)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)

    if path is not None:
        for i in range(path.shape[0]-1): # iterate through steps
            if i%skip==0:
                # only plot arrows every "skip" iterations
                plt.arrow(path[i,0],path[i,1],path[i+1,0]-path[i,0],path[i+1,1]-path[i,1],
                          color='k',length_includes_head=True)
    plt.show()

In [ ]:
plot_path_Beale(None,g,'Beale function') # plot with no path to just show contour plot

First, let's explore how regular steepest descent performs on this function. One way we could attempt to determine an optimal step size for this function would be to determine its Hessian and see if we can bound the eigenvalues. Another (lazier but much faster) method might just be to simply evaluate (or even approximate) the Hessian only at the known minimizer $(3,0.5)$ and determine eigenvalues. Let's do that one, the easier one!

In [ ]:
d2g = lambda x, y: np.array([[0.5*(dg(x+h, y)[0]-dg(x-h, y)[0])/h, 0.5*(dg(x, y+h)[0]-dg(x, y-h)[0])/h],
                             [0.5*(dg(x+h, y)[1]-dg(x-h, y)[1])/h, 0.5*(dg(x, y+h)[1]-dg(x, y-h)[1])/h]])

d2g(3,0.5)

In [ ]:
eigs, _ = np.linalg.eigh(d2g(3,0.5))
print(eigs)

print('alpha = ',4/(np.sqrt(eigs[0])+np.sqrt(eigs[1]))**2)
print('beta = ',(np.sqrt(eigs[1])-np.sqrt(eigs[0]))**2/(np.sqrt(eigs[1])+np.sqrt(eigs[0]))**2)

Let's try this step size below, first with steepest descent, then do the same thing using heavyball, both starting from the initial point $(-3,-1)$:

In [ ]:
%%time
x = np.array([-3,-1])
path_SD1_Beale = [x]
tol = 1e-8
alpha = 4/(np.sqrt(eigs[0])+np.sqrt(eigs[1]))**2   # optimal(?) step size
max_steps = 20000
dx = dg(x[0],x[1])
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    xnew = x - alpha*dx
    path_SD1_Beale.append(xnew)
    x = xnew
    i += 1
    dx = dg(x[0],x[1])

path_SD1_Beale = np.array(path_SD1_Beale)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')

In [ ]:
plot_path_Beale(path_SD1_Beale,g,'Steepest descent on Beale function, first attempt')

Well that didn't work. Does heavyball do any better?

In [ ]:
%%time
x = np.array([-3,-1])
path_HB1_Beale = [x]
tol = 1e-8
beta = (np.sqrt(eigs[1])-np.sqrt(eigs[0]))**2/(np.sqrt(eigs[1])+np.sqrt(eigs[0]))**2  # optimal(?) momentum parameter
alpha = 4/(np.sqrt(eigs[0])+np.sqrt(eigs[1]))**2                                      # optimal(?) step size
max_steps = 1000
dx = dg(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    pk = -alpha*dx + beta*pk
    xnew = x + pk
    path_HB1_Beale.append(xnew)
    x = xnew
    i += 1
    dx = dg(x[0],x[1])

path_HB1_Beale = np.array(path_HB1_Beale)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')

In [ ]:
plot_path_Beale(path_HB1_Beale,g,'Heavyball on Beale function, first attempt')

[That's a negative, ghost rider; the pattern is full.](https://i.imgur.com/NlV7ISw.jpeg)

Maybe a different value of $\alpha$ and $\beta$ would be better? Rather than just guessing (which tbh is the most commonly employed strategy), let's still try to stick to some semblance of rigor and do a [Monte Carlo approximation](https://en.wikipedia.org/wiki/Monte_Carlo_method) of the largest and smallest eigenvalues in our viewing window of $[-4.5,4.5]\times[-4.5,4.5]$:

In [ ]:
maxval = eigs[1]
minval = eigs[0]
for i in range(1000):
    x = np.random.uniform(-4.5,4.5,2) # generate a random point
    H = d2g(x[0],x[1])                # and evaluate the Hessian there
    tmp_eigs, _ = np.linalg.eigh(H)
    if tmp_eigs[1]>maxval:                # if eigenvalues larger or smaller, choose those
        maxval = tmp_eigs[1]
    if tmp_eigs[0]<minval:
        minval = tmp_eigs[0]
        
print('Maximum eigenvalue :',maxval,'\nMinimum eigenvalue :',minval)

Since there are some *concave* points in our domain, we have negative eigenvalues. Promptly ignoring that, let's simply use the maximum eigenvalue from above as part of our calculations, keeping the smaller eigenvalue from the minimizer:

In [ ]:
%%time
x = np.array([-3,-1])
path_SD2_Beale = [x]
tol = 1e-8
alpha = 4/(np.sqrt(eigs[0])+np.sqrt(maxval))**2   # optimal(?) step size
print('alpha = ',alpha)
max_steps = 20000
dx = dg(x[0],x[1])
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    xnew = x - alpha*dx
    path_SD2_Beale.append(xnew)
    x = xnew
    i += 1
    dx = dg(x[0],x[1])

path_SD2_Beale = np.array(path_SD2_Beale)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')

In [ ]:
plot_path_Beale(path_SD2_Beale,g,'Steepest descent on Beale function, second attempt', skip=10)

Well at least we're moving now, but 1) This is taking way too long, and 2) We don't appear to be converging to the right place! How about heavyball?

In [ ]:
%%time
x = np.array([-3,-1])
path_HB2_Beale = [x]
tol = 1e-8
beta = (np.sqrt(maxval)-np.sqrt(eigs[0]))**2/(np.sqrt(maxval)+np.sqrt(eigs[0]))**2  # optimal(?) momentum parameter
print('beta = ',beta)
alpha = 4/(np.sqrt(eigs[0])+np.sqrt(maxval))**2                                      # optimal(?) step size
print('alpha = ',alpha)
max_steps = 20000
dx = dg(x[0],x[1])
pk = dx*0      # first update is just gradient descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    pk = -alpha*dx + beta*pk
    xnew = x + pk
    path_HB2_Beale.append(xnew)
    x = xnew
    i += 1
    dx = dg(x[0],x[1])

path_HB2_Beale = np.array(path_HB2_Beale)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')

In [ ]:
plot_path_Beale(path_HB2_Beale,g,'Heavyball on Beale function, second attempt', skip=10)

Hey! We finally converged! This ad hoc method of choosing $\alpha$ and $\beta$ actually worked! Actually, if you investigate the path above, though, it appears we were already close to the minimum within only 1000 steps; the last 9000+ steps were all jostling around inside the valley. Perhaps we can modify the value of $\alpha$ or $\beta$ to get better convergence?

**Exericise:** Try that.

Finally, let's look at the nonlinear conjugate method with an FR update. Although FR should use an adaptive $\alpha$, let's try conjugate gradient with a fixed step size to see a somewhat fair comparison:

In [ ]:
%%time
x = np.array([-3,-1])
path_FR1_Beale = [x]
tol = 1e-8
alpha = 4/(np.sqrt(eigs[0])+np.sqrt(maxval))**2
max_steps = 20000
dx = dg(x[0],x[1])
pk = -dx    # first step is steepest descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    xnew = x + alpha*pk
    dx1 = dg(xnew[0],xnew[1])
    bk = FR_beta(dx,dx1)    # still use adaptive beta
    if i>0 and i%3 ==0:
        bk = 0    # restarting
    pk = -dx1 + bk*pk

    path_FR1_Beale.append(xnew)
    x = xnew
    dx = dx1
    i += 1

path_FR1_Beale=np.array(path_FR1_Beale)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')

In [ ]:
plot_path_Beale(path_FR1_Beale,g,'FR on Beale function, fixed step size', skip=10)

We see that with the same fixed step size, even if $\beta$ is calculated adaptively, conjugate gradient does not perform well, indeed about the same as steepest descent. However, below, we unleash the full power of conjugate gradient, allowing both $\alpha$ and $\beta$ to be calculated adaptively, here by using backtracking on alpha:

In [ ]:
def WolfeI(alpha,f,x,dx,p,c1=0.1):
    '''Return True/False if Wolfe condition I is satisfied for the given alpha'''
    LHS = f(x[0]+alpha*p[0], x[1]+alpha*p[1])
    RHS = f(x[0],x[1])+c1*alpha*np.dot(dx,p)
    return LHS <= RHS

In [ ]:
%%time
x = np.array([-3,-1])
path_FR2_Beale = [x]
tol = 1e-8
rho = 0.9  # rho for backtracking
max_steps = 10000
dx = dg(x[0],x[1])
pk = -dx    # first step is steepest descent
i=0
while np.linalg.norm(dx)>tol and i<max_steps:
    # backtracking
    alpha = 1
    while not WolfeI(alpha,g,x,dx,pk) and alpha>1e-5: # lower bound on alpha attempts to force Wolfe II
        alpha *= rho
        
    xnew = x + alpha*pk
    dx1 = dg(xnew[0],xnew[1])
    bk = FR_beta(dx,dx1)
    if i>0 and i%3 ==0:
        bk = 0    # restarting
    pk = -dx1 + bk*pk

    path_FR2_Beale.append(xnew)
    x = xnew
    dx = dx1
    i += 1

path_FR2_Beale=np.array(path_FR2_Beale)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')

In [ ]:
plot_path_Beale(path_FR2_Beale,g,'FR on Beale function, adaptive step size')

I recommend playing around with the different parameters above to see how they affect convergence. What happens if we change $\rho$ in backtracking? What about $c_1$ in the Wolfe I condition? Especially with such a non-convex function, this is an open question. There are no rules or expected results; simply exploration.